# Get Data from Airtable

This gets the data from the two tables and formats it into dataframes. It's probably not the cleanest thing in the world, but hopefully it'll do for now.

In [53]:
import ast
import pandas as pd
import requests
from graphviz import Source
import textwrap
import simplejson

In [54]:
def mergeD(ugly_data):
    f = ugly_data["fields"]
    i = {"id": ugly_data["id"]}
    f.update(i)
    return f

def get_table(table_name):
    """ """
    auth_headers = {'Authorization': 'Bearer keyYXkjYFw61SeWDk'}
    table_url = 'https://api.airtable.com/v0/appuaXpFiadmP89sq/' + table_name
    r = requests.get(table_url, headers=auth_headers)
    d = eval(ast.literal_eval(str(r.content)[1:])) # this is almost certainly not the right way to do this!
    
    rows = [mergeD(x) for x in d["records"]]
    data_frame = pd.DataFrame(rows)
    return data_frame

In [64]:
project_data = get_table("Techs")
project_data.set_index("id", inplace=True)

node_description_pattern = "{name} [label=\"{longName}\"];"
edge_pattern = "{from_node} -> {to_node};"

node_descriptions = []
edges = []
for i, row in project_data.iterrows():
    try:
#         ln = textwrap.fill(row["LongName"], 21)
        ln = "\n".join( textwrap.wrap(row["LongName"], 15))
    except Exception as e:
        ln = row["LongName"]
    n = node_description_pattern.format(name=row["Name"], longName=ln)
    node_descriptions.append(n)
    if type(row["precursors"]) is not float: # nothing reads as NaN, therefore float
        precursors = row["precursors"]#.split(",")
        for precursor in precursors:
            e = edge_pattern.format(from_node=project_data.loc[precursor].Name,
                                    to_node=row["Name"])
            edges.append(e)

g = """digraph G {{
  rankdir  = "LR";
  label    = "Tech Tree of OATMEAL Projects";
  labelloc = "t";
  node [style=filled,
        color="black", 
        fontcolor="white", 
        shape="oval", 
        fixedsize=true, 
        width=1,
        height=1,
        fontsize=8 
       ];
  edge [arrowhead=vee,
        arrowsize=0.5]
  
  {nodes}
  
  {edges}
  {{rank = same; data; basicRobot; hardware; systemReef; SNAawareness}}
  
}}""".format(edges="\n  ".join(edges), nodes="\n  ".join(node_descriptions))


# print(g)
g.split("\n ")

"""var digraph_details = {};

digraph_details =  digraph_details.join("\n");"""

['digraph G {',
 ' rankdir  = "LR";',
 ' label    = "Tech Tree of OATMEAL Projects";',
 ' labelloc = "t";',
 ' node [style=filled,',
 '       color="black", ',
 '       fontcolor="white", ',
 '       shape="oval", ',
 '       fixedsize=true, ',
 '       width=1,',
 '       height=1,',
 '       fontsize=8 ',
 '      ];',
 ' edge [arrowhead=vee,',
 '       arrowsize=0.5]',
 ' ',
 ' potPaper [label="A Plane of\nThrones Paper"];',
 ' robPaper [label="Robotics Papers"];',
 ' deskSenseDn [label="Below Desk\nSensing"];',
 ' desks [label="Unit Desks"];',
 ' deskSenseUp [label="Above Desk\nSensing"];',
 ' mApp [label="Mobile App"];',
 ' geoEthno2 [label="Geographic/\nEthnographic\ndata 2"];',
 ' basicBeacon [label="iBeacon\nAwareness"];',
 ' sensicorn2 [label="Sensicorn 2"];',
 ' geoEthno1 [label="Geographic/\nEthnographic\ndata 1"];',
 ' emailData [label="Email Data"];',
 ' RBTM [label="Robotic\nBuilding\nTopology\nModification"];',
 ' basicRobot [label="Awareness of\nRobots"];',
 ' timeData [

In [65]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_dict = project_data.to_dict(orient="records")
new_dict = {}
for row in simple_dict:
    new_dict[row["Name"]] = row

with open('parallelData.js', 'w') as f:
    j = simplejson.dumps(new_dict, ignore_nan=True, sort_keys=True, indent=2)
    f.write("var parallelData = {json};".format(json=j))

In [57]:
def get_thumb(row):
    try:
        h = row["Headshot"]
        if type(h) is not float:
            # TODO: make this check for small as well
            x = list(h)
            return x[0]['thumbnails']["large"]["url"]
        else:
            return "http://getdrawings.com/images/casper-drawing-8.jpg"
    except Exception as e:
        print(e)
        return "http://getdrawings.com/images/casper-drawing-8.jpg"


In [58]:

people_data = get_table("People")
people_data["thumb_large"] = people_data.apply(get_thumb, axis=1)
people_data

,Headshot,Initials,Name,Notes,Techs,id,thumb_large
0,"[{'thumbnails': {'large': {'height': 500, 'url...",KM,Kevin M,NaN,[recnXgQGgzh4q3t9M],rec0dpxUhHfgHWU7r,https://dl.airtable.com/6ax7C3bmSUWbWYULMZsZ_l...
1,"[{'thumbnails': {'large': {'height': 500, 'url...",ARR,Annisa,NaN,[recru4UXYdMmO7sX8],rec3HypnJiuTkeVBJ,https://dl.airtable.com/JgeHAo9xSGmjVWy0FtXS_l...
2,"[{'thumbnails': {'large': {'height': 512, 'url...",--,nobodyAssigned,This is who'll be assigned automagically if no...,NaN,recF9tgC6LJUmi5gJ,https://dl.airtable.com/JDhVYh9SAOPYoIBE6ipb_l...
3,"[{'thumbnails': {'large': {'height': 500, 'url...",BaD,Barry D,NaN,NaN,recFKWrjwOUM5oWxI,https://dl.airtable.com/4NWLSnTFRwSfdFDBAt9A_l...
4,NaN,RX,AI Richard,NaN,NaN,recFUgthTUYQnR7Mz,http://getdrawings.com/images/casper-drawing-8...
5,"[{'thumbnails': {'large': {'height': 500, 'url...",NT,Ninotschka,NaN,[rec0tbY526abDXcRr],recHLT6IoCZXwwtp3,https://dl.airtable.com/n5K9IIPLSEiPPcY8qKCG_l...
6,"[{'thumbnails': {'large': {'height': 500, 'url...",BiD,Bill,NaN,"[recmNHHmIUj4nPild, recHpePW60ZlXrLKY]",recHN7P3Wz3YGdAN2,https://dl.airtable.com/nestrTYmQqT79f6rfpHQ_l...
7,"[{'thumbnails': {'large': {'height': 500, 'url...",BC,Barry C,NaN,[recqsdnZyKEXLlafN],recMZdSRhVMMJQcOb,https://dl.airtable.com/tEyLZYnrSSaN4Ikic0k5_l...
8,"[{'thumbnails': {'large': {'height': 683, 'url...",BeD,Ben,NaN,"[reccHCPeML8SyRFTp, rec5v7YBAh4XzeOQ7, recSIIp...",rece9qyNSfMqAhZAL,https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_l...
9,"[{'thumbnails': {'large': {'height': 831, 'url...",IV,Ishaan,NaN,"[recCdV0IBu62AhiOt, recmjo5fVtiAflmvo, recflCm...",recgcDkgKvrBLH1k0,https://dl.airtable.com/hI2scvcS1WedienZx2uw_l...


In [59]:
# make the dictionary have the project name as a key, instead of bing a list of independent dicts
simple_people_dict = people_data.to_dict(orient="records")
new_p_dict = {}
for row in simple_people_dict:
    new_p_dict[row["id"]] = row
    
new_p_dict


with open('peopleData.js', 'w') as f:
    j = simplejson.dumps(new_p_dict, ignore_nan=True, sort_keys=True, indent=2)
    f.write("var peopleData = {json};".format(json=j))

Aiming for something that looks a bit like this:
![](https://lh3.googleusercontent.com/xm3KUhGDiEDbr5v9jnlTv3kqqBDbTXP497VxE2otSsmHMpdDi5kBZaBnCnWCzknNFZynJw91wm8fRoOS_VqJCuQOA_4dCXYrqkIQG_YHMwmuNsqiPBJnAVRu1e1ZMvywiw9JkQDkCJM=w1430-h1015-no)
So we need these bits of information:

In [60]:
this_project = project_data[project_data.Name == "deskSenseUp"] # "AIlayout" for my face
print(this_project.LongName)
owner = list(this_project.Owner)[0][0]
this_project

id
rec5v7YBAh4XzeOQ7    Above Desk Sensing
Name: LongName, dtype: object


,LongName,Name,Other_people_involved,Owner,Type,percentComplete,precursors
id,,,,,,,
rec5v7YBAh4XzeOQ7,Above Desk Sensing,deskSenseUp,"[recuFyI25ChOwWWhr, rece9qyNSfMqAhZAL]",[recj2rmnndUmxB3Dt],Project,50,"[recqsdnZyKEXLlafN, rechAwGrx1K8hAmTi]"


In [61]:
list(people_data[people_data.id == owner].thumb_large)

['https://dl.airtable.com/wNAkSR0RUlLTf8AMRYJA_large_50.png']

In [62]:
from IPython.display import Image
from IPython import display

halpers = this_project.Other_people_involved.values[0]
for h in halpers:
    row = people_data[people_data.id == h]
    print(h, row.Name.values[0], row.thumb_large.values[0])
    display.display(row.thumb_large.values[0])
#     Image()

recuFyI25ChOwWWhr Baptiste https://dl.airtable.com/FRJXAfziQn226JiBopNq_large_20171005_171412_resized.jpg


'https://dl.airtable.com/FRJXAfziQn226JiBopNq_large_20171005_171412_resized.jpg'

rece9qyNSfMqAhZAL Ben https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_large_MVIMG_20180224_173642.jpg


'https://dl.airtable.com/nUBlmdvR1iLT0mCbllA1_large_MVIMG_20180224_173642.jpg'

In [63]:
#now this is ready to drive the progress ring
list(this_project.percentComplete)[0]#.replace("%", "")

50

![](https://dl.airtable.com/vao3Ge65RXuQs17gLU4p_small_MVIMG_20180224_173642.jpg)